In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import rawPy as rp

In [2]:
#Choose the preferred backend for interactive plotting
%matplotlib qt

##########################  
**Date Run:** 2/3/18 Scuderi   
**Date Reduced:** 3/7/18 Scuderi  
##########################  
**Material:** Wenchuan Drillin project WFSD-3P-6  
**Configuration:** Double-Direct Shear 5x5 5x5 small grooves side blocks  
**Normal stress:** 12MPa  
**Shear velocity:** 10µm/s  
**Velocity steps:** 0.3µm/s-1mm/s  
**Slide-hold-slide:** 10-10000  
**Notes:**  
W1 = 8.24g  
W2 = 8.28g  

In [3]:
#LOAD DATA
exp_name = 'b754s03wen6_12'
data = rp.load_data('%s'%exp_name)

------------------------------------------------------
|                Name|           Unit|        Records|
-----------------------------------------------------
|              V-LVDT|           (mm)|         223102|
|              H-LVDT|           (mm)|         223102|
|              V-LOAD|           (kN)|         223102|
|              H-LOAD|           (kN)|         223102|
|                Time|          (sec)|         223102|
|              Record|            (.)|         223102|
------------------------------------------------------


#### 1. Calculate Normal Stress ####

In [5]:
# PLOT
rp.plot(data.Record,data['H-LOAD'],'rec#','h_load')

In [11]:
#############  normal load ############ 
#### Normal load is applied at record

beg_row_h = 342

######################################
# zero the data
h_load = rp.zero(data['H-LOAD'],beg_row_h)
# remove noise before load is applied 
h_load[:beg_row_h] = h_load[:beg_row_h]*0
# calculate stress for DDS 5x5
h_load =  h_load/(1000*0.0025)
#add a small number to normal stress 
#so that we aren't dividing by 0 anywhere to calculate mu 
normal_stress_MPa = h_load + 1e-7
del h_load

rp.plot(data.Time,normal_stress_MPa,'Time [s]','Normal stress [MPa]')

#### 2. Calculate Shear Stress ####

In [12]:
# PLOT
rp.plot(data.Record,data['V-LOAD'],'rec#','v_load [kN]')

In [13]:
#### Vertical stress ###
#### This is the point at which the ram contacted the blocks and load shear loading began

beg_row_v = 7625

##################################

# ZERO DATA
v_load = rp.zero(data['V-LOAD'],beg_row_v)
# add in force from the central block due to gravity 
v_load = v_load + 0.044 #[kN]
# remove noise before load is applied
v_load[:beg_row_v]= v_load[:beg_row_v]*0
# calculate stress for DDS 5x5
shear_stress_MPa = v_load/(2*1000*0.0025)
del v_load

#### vertical disp ####
lp_disp_mm = rp.zero(data['V-LVDT'],beg_row_v)
lp_disp_mm[:beg_row_v] = lp_disp_mm[:beg_row_v]*0
lp_disp_mm = lp_disp_mm*-1

rp.plot(data.Time,shear_stress_MPa,'Time [s]','Shear stress [MPa]')
rp.plot(data.Time,lp_disp_mm,'Time [s]', 'lp_disp [mm]')

**If necessary offset the data**

In [9]:
# OFFSET data 
# takes as input row1, row2, col 
#example
#row_1 = 89726
#row_2 = 89737
#lp_disp = rp.offset(lp_disp,row_1,row_2)
#

#### 3. Correct for elastic strech of the vertical frame ####  

| Verical stiffness | applied load | k[MPa/mm] | calibration date | 
| --- | --- | --- | --- |  
| 359.75 [kN/mm] | < 50 [kN] | 116.801 [MPa/mm] | 19/11/2015 | 
| 928.5 [kN/mm] | > 50 [kN] | 301.461 [MPa/mm] | 19/11/2015 | 

Note:  
1. These values are calculated for a 5x5[cm] shear surface  
2. Stiffness is non linear at low applied loads  
TO DO: implement a function that corrects for non-linear stiffness


In [15]:
# Elastic correction for the stretch of the vertical frame # 
k = 166.801 #[MPa/mm]
ec_disp_mm = rp.ElasticCorrection(shear_stress_MPa,lp_disp_mm,k)

fig = plt.figure()
plt.plot(data.Time,ec_disp_mm,'r', label='ec_disp')
plt.plot(data.Time,lp_disp_mm,'b', label='lp_disp')
plt.xlabel('Time [s]')
plt.ylabel('Disp [mm]')
plt.legend(loc='upper right')

#### 4. Correct for elastic strech of the horizontal frame ####  

| Horizontal stiffness | applied load | k[MPa/mm] | calibration date | 
| --- | --- | --- | --- |  
| 386.12 [kN/mm] | < 50 [kN] | 125.363 [MPa/mm] | 19/11/2015 | 
| 1283 [kN/mm] | > 50 [kN] | 416.558 [MPa/mm] | 19/11/2015 | 

Note:  
1. These values are calculated for a 5x5[cm] shear surface  
2. Stiffness is non linear at low applied loads  
TO DO: implement a function that corrects for non-linear stiffness


In [20]:
# Elastic correction for the stretch of the horizontal frame #
k = 125.363 #[MPa/mm]
lt_ec_mm = rp.ElasticCorrection(normal_stress_MPa,data['H-LVDT'],k)

### 5. Calculate layer thickness ###  

Treat changes in horizontal displacement ($\Delta$h) as symmetric, take half of it for 1 layer.  
Compaction = thinner layer  
Thickness of DDS assembly with no gouge is:  

| small groove blocks | large groove blocks | PZT side blocks |  
| --- | --- | --- |  
|99.7 mm | 95 mm | 102.54 mm |

Bench thickness of initial layers & DDS is  --- [mm]  
Bench thickness of 1 layer is: ---  [mm]

Thickness of spacers:  

| AA | BB | CC |  
| --- | --- | --- |  
|29.86 mm | -- mm | 6.12 mm |

The layer thickness is calculated as:  
layer_thickness = (total layer thickness)-(Assembly)-(Spacers) / 2  


Total thickness is 137.14 mm at rec #6068  
layer tickness is  (137.14- 99.7 -29.86-6.12)/2  
Layer thickness for one layer under load is 0.73 mm  

In [21]:
###########################################################
# insert rec number at which layer thickness was measured
rec_lt = 6068
# insert calculated value at that poin
val_lt = 0.73
###########################################################
# zero data
lt_ec_mm = rp.zero(lt_ec_mm,rec_lt)
lt_ec_mm = lt_ec_mm/2
lt_ec_mm = lt_ec_mm + val_lt

rp.plot(data.Time,lt_ec_mm,'Time [s]','Layer thickness [mm]')

### 6. Calculate shear strain ###

In [22]:
##### calculate strain ####
shear_strain_na = rp.shear_strain(ec_disp_mm,lt_ec_mm)

rp.plot(shear_strain_na,shear_stress_MPa,'Shear strain','Shear stress [MPa]')

### 7. Calculate friction ###

In [23]:
# select last row of experiment for friction
rp.plot(data.Record,shear_stress_MPa,'rec','shear stress')

In [24]:
##### insert the last row 
end_row = 208901
########################

friction_na = shear_stress_MPa/normal_stress_MPa
# remove the noise before and after vertical load
friction_na[:beg_row_v] = friction_na[:beg_row_v]*0
friction_na[end_row:] = friction_na[end_row:]*0

rp.plot(ec_disp_mm,friction_na,'ec_disp [mm]','friction')

### 8. Export reduced data ###

In [25]:
time_s = data.Time
records_na = data.Record # record must always be the last col

data_out = [shear_stress_MPa,
            lp_disp_mm,
            ec_disp_mm,
            normal_stress_MPa,
            lt_ec_mm,
            friction_na,
            shear_strain_na,
            time_s,
            records_na]
            
rp.save_data(exp_name,data_out,callingLocals=locals())

Saving data

------------------------------------------------------
|                Name|           Unit|        Records|
-----------------------------------------------------
|    shear_stress_MPa|            MPa|         223103|
|          lp_disp_mm|             mm|         223103|
|          ec_disp_mm|             mm|         223103|
|   normal_stress_MPa|            MPa|         223103|
|            lt_ec_mm|             mm|         223103|
|         friction_na|             na|         223103|
|     shear_strain_na|             na|         223103|
|              time_s|              s|         223103|
|          records_na|             na|         223103|
------------------------------------------------------


In [26]:
## import the data analyzed
new_data = rp.load_data('b754_data_new')

------------------------------------------------------
|                Name|           Unit|        Records|
-----------------------------------------------------
|    shear_stress_MPa|            MPa|         223102|
|          lp_disp_mm|             mm|         223102|
|          ec_disp_mm|             mm|         223102|
|   normal_stress_MPa|            MPa|         223102|
|            lt_ec_mm|             mm|         223102|
|         friction_na|             na|         223102|
|     shear_strain_na|             na|         223102|
|              time_s|              s|         223102|
|          records_na|             na|         223102|
------------------------------------------------------


In [27]:
## import the data analyzed as pandas data frame
new_data = rp.load_data('b754_data_new',pandas=True)
new_data

------------------------------------------------------
|                Name|           Unit|        Records|
-----------------------------------------------------
|    shear_stress_MPa|            MPa|         223102|
|          lp_disp_mm|             mm|         223102|
|          ec_disp_mm|             mm|         223102|
|   normal_stress_MPa|            MPa|         223102|
|            lt_ec_mm|             mm|         223102|
|         friction_na|             na|         223102|
|     shear_strain_na|             na|         223102|
|              time_s|              s|         223102|
|          records_na|             na|         223102|
------------------------------------------------------


,shear_stress_MPa,lp_disp_mm,ec_disp_mm,normal_stress_MPa,lt_ec_mm,friction_na,shear_strain_na,time_s,records_na
0,-0.000000,0.000000,0.000000,1.000000e-07,2.805248,-0.0,0.000000,0.0,0.0
1,-0.000000,0.000000,0.000000,1.000000e-07,2.805240,-0.0,0.000000,0.1,1.0
2,-0.000000,0.000000,0.000000,1.000000e-07,2.805238,-0.0,0.000000,0.2,2.0
3,-0.000000,0.000000,0.000000,1.000000e-07,2.805233,-0.0,0.000000,0.3,3.0
4,-0.000000,0.000000,0.000000,1.000000e-07,2.805247,-0.0,0.000000,0.4,4.0
...,...,...,...,...,...,...,...,...,...
223098,0.059976,22.010925,22.010565,8.294210e-02,0.644625,0.0,52.572861,24783.0,223098.0
223099,0.060390,22.010940,22.010578,8.286370e-02,0.644623,0.0,52.572881,24784.0,223099.0
223100,0.060594,22.010916,22.010553,8.238050e-02,0.644628,0.0,52.572841,24785.0,223100.0
223101,0.060607,22.010929,22.010566,8.194650e-02,0.644628,0.0,52.572861,24786.0,223101.0
